<a href="https://colab.research.google.com/github/oscarserc/TESIS/blob/main/PASO_3b___CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#UTILIZACIÓN DE TÉCNICAS DE APRENDIZAJE AUTOMÁTICO PARA LA DETECCIÓN DE LOS NODOS INFLUYENTES.

La base de datos que emplearemos será de unas 70.000 observaciones. En concreto:

· 72.000 observaciones

· Para ello generamos una 1800 **simulaciones**, y guardamos unos 40 **frames** de cada una de ellas

· Además como **target** guardamos la matriz con la IDENTIDAD de todos y cada uno de los AGENTES.



#**PRUEBA 2**: Utilizaremos una <font color='green'>**RED NEURONAL CONVOLUCIONAL**</font>. Planteamos el problema de localizar a los agentes influyentes como un problema de **Segmentación** de imágenes.


<figure>
<center>
<img src='https://github.com/oscarserc/IMAGENES/blob/main/Archivo_D.png?raw=true' width="950"/>
<figcaption></figcaption></center>
</figure>

